In [ ]:
# Config

import os
import subprocess
import glob
%env DATADIR = "/home/s/Documents/data/ossExperiment"
DATADIR = os.environ.get("DATADIR")[1:-1]


In [ ]:
#benchmarks
class Benchmark:
    def __init__(self, name,url,commitHash,primaryApk,pkg):
        self.name = name
        self.url = url
        self.commitHash = commitHash
        # self.print = lambda x:x
        self.print = print
        self.workDir = DATADIR + "/" + self.name
        self.primaryApk = primaryApk
        self.pkg = pkg
    def clone(self):
        #subprocess.Popen(["touch","abcdefghij"], cwd=DATADIR)
        cmd = ["git", "clone", self.url]
        self.print("cloning: %s -- %s" % (self.url,str(cmd)))
        p1 = subprocess.Popen(["git", "clone", self.url],cwd=DATADIR,stdout=subprocess.PIPE)
        output,error = p1.communicate()
        self.print("output:\n %s" % output)
        self.print("error:\n %s" % error)
        self.print("checking out hash: %s" % self.commitHash)
        p2 = subprocess.Popen(["git", "checkout", self.commitHash])
        output,error = p2.communicate()
        self.print("output:\n %s" % output)
        self.print("error:\n %s" % error)
    def build(self):
        p2 = subprocess.Popen(["jenv", "local", "13.0.1"])
        p2.communicate()
        output,error = p2.communicate()
        self.print("output:\n %s" % output)
        self.print("error:\n %s" % error)
        cmd = ["./gradlew","assembleDebug"]
        p = subprocess.Popen(cmd, cwd=self.workDir, stdout=subprocess.PIPE)
        output,error = p.communicate()
        self.print("output:\n %s" % output)
        self.print("error:\n %s" % error)
    def findApk(self):
        foundApks = glob.glob("%s/**/*.apk" % self.workDir,recursive=True)
        return [f for f in foundApks if self.primaryApk in f][0]
        
        
benchmarks = [ 
    Benchmark('AntennaPod','https://github.com/AntennaPod/AntennaPod.git','88289d0','app-free-debug', 'de.danoeh.antennapod'), 
    Benchmark('connectbot','https://github.com/connectbot/connectbot.git','7b6973f','app-google-debug', 'org.connectbot'), 
    Benchmark('sgtpuzzles','https://github.com/chrisboyle/sgtpuzzles.git','83d3859','app-debug', 'name.boyle.chris')]

In [ ]:
f = open('benchmark-list.txt','w')
for b in benchmarks:
    # b.clone()
    # b.build()
    
    benchmarkPkg = b.pkg
    benchmarkApk = b.findApk()
    f.write("%s:%s\n" % (benchmarkApk,benchmarkPkg))
    
f.close()